### CNN - using GPU

In [7]:
# import libraries
import torch
from torch.optim import Adam
from torch.nn import Linear, Sigmoid, BCELoss, Module, Softmax, ReLU, CrossEntropyLoss, Conv2d, MaxPool2d
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

# import MNIST data
from tensorflow.keras.datasets.mnist import load_data

In [8]:
# move model to GPU if available
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# load data
(x_train, y_train), (x_test, y_test) = load_data()

In [11]:
# reshape to (60000, 1, 28), normalization, tensor node
x = torch.FloatTensor(x_train.reshape(-1, 1, 28, 28) / 255)
y = torch.LongTensor(y_train)

In [ ]:
# transfer feature and label to TensorDataset 
dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)
print(len(dataloader)) # 60000 / 256

235


In [23]:
# CNN Class
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2d(in_channels=1, out_channels=3, kernel_size=5, stride=1)
        self.conv2 = Conv2d(in_channels=3, out_channels=10, kernel_size=5, stride=1)
        self.max1 = MaxPool2d(kernel_size=2, stride=1)
        self.fc1 = Linear(10 * 19 * 19, 36)
        self.fc2 = Linear(36, 10)
        self.smax = Softmax(dim=1)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.max1(x)
        x = x.view(-1, 10 * 19 * 19)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.smax(x)
        return x

In [ ]:
# training
model = CNN().to(DEVICE)
loss_fn = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

for epoch in range(20):
    print(f'Epoch [{epoch+1}/20]')
    
    for batch_x, batch_y in dataloader:
        batch_x = batch_x.to(DEVICE)
        batch_y = batch_y.to(DEVICE)

        optimizer.zero_grad()
        hx = model(batch_x)
        cost = loss_fn(hx, batch_y)
        cost.backward()
        optimizer.step()

        _, pred = hx.max(dim=1)
        pred_cpu = pred.cpu().numpy()
        batch_y_cpu = batch_y.cpu().numpy()

        print(f"    Loss: {cost.item():.4f} | Accuracy: {accuracy_score(batch_y_cpu, pred_cpu) * 100:.2f}%")

Epoch [1/20]
    Loss: 2.3030 | Accuracy: 6.64%
    Loss: 2.2927 | Accuracy: 14.06%
    Loss: 2.2922 | Accuracy: 12.11%
    Loss: 2.2208 | Accuracy: 22.66%
    Loss: 2.1613 | Accuracy: 29.30%
    Loss: 2.1025 | Accuracy: 41.80%
    Loss: 1.9982 | Accuracy: 50.78%
    Loss: 1.9672 | Accuracy: 50.78%
    Loss: 1.8233 | Accuracy: 65.62%
    Loss: 1.8701 | Accuracy: 58.59%
    Loss: 1.8109 | Accuracy: 66.02%
    Loss: 1.7670 | Accuracy: 70.70%
    Loss: 1.7698 | Accuracy: 69.53%
    Loss: 1.7398 | Accuracy: 72.27%
    Loss: 1.7248 | Accuracy: 73.83%
    Loss: 1.7666 | Accuracy: 69.53%
    Loss: 1.7334 | Accuracy: 73.05%
    Loss: 1.7643 | Accuracy: 69.14%
    Loss: 1.7248 | Accuracy: 73.44%
    Loss: 1.7613 | Accuracy: 69.92%
    Loss: 1.6987 | Accuracy: 76.17%
    Loss: 1.7361 | Accuracy: 72.27%
    Loss: 1.7351 | Accuracy: 72.66%
    Loss: 1.6649 | Accuracy: 79.30%
    Loss: 1.7172 | Accuracy: 75.00%
    Loss: 1.7383 | Accuracy: 71.88%
    Loss: 1.7090 | Accuracy: 75.39%
    Loss: 1.7016